### Jan 28

-  Created the API wrapper for one trial

In [2]:
ct_get_study_module_table <- function(nct_id, module = NULL) {
  # deps
  if (!requireNamespace("httr2", quietly = TRUE)) stop("Install httr2")
  if (!requireNamespace("jsonlite", quietly = TRUE)) stop("Install jsonlite")
  if (!requireNamespace("tibble", quietly = TRUE)) stop("Install tibble")

  if (!is.character(nct_id) || length(nct_id) != 1 || !nzchar(nct_id)) {
    stop("`nct_id` must be a single non-empty character string like 'NCT04267848'.")
  }
  if (!is.null(module) && (!is.character(module) || length(module) != 1 || !nzchar(module))) {
    stop("`module` must be NULL or a single non-empty character string like 'statusModule'.")
  }

  # --- helper: check "empty" ---
  is_empty <- function(x) {
    is.null(x) ||
      length(x) == 0 ||
      (is.character(x) && all(!nzchar(x))) ||
      (is.list(x) && length(x) == 0)
  }

  # --- helper: flatten nested lists into a named list of leaf scalars ---
  flatten_any <- function(x, prefix = "") {
    out <- list()
    if (is_empty(x)) return(out)

    # atomic leaf
    if (is.atomic(x) && !is.list(x)) {
      val <- x
      if (length(val) > 1) val <- paste(as.character(val), collapse = " | ")
      else val <- as.character(val)[1]
      if (!is_empty(val)) out[[prefix]] <- val
      return(out)
    }

    # list
    if (is.list(x)) {
      # named list
      if (!is.null(names(x)) && any(nzchar(names(x)))) {
        nms <- names(x)
        for (i in seq_along(x)) {
          nm <- nms[i]
          if (is.na(nm) || !nzchar(nm)) next
          key <- if (nzchar(prefix)) paste0(prefix, ".", nm) else nm
          out <- c(out, flatten_any(x[[i]], key))
        }
        return(out)
      }

      # unnamed list (array-like)
      all_scalar <- all(vapply(x, function(z) is.atomic(z) && !is.list(z), logical(1)))
      if (all_scalar) {
        vals <- unlist(lapply(x, as.character), use.names = FALSE)
        vals <- vals[nzchar(vals)]
        if (length(vals) > 0) out[[prefix]] <- paste(vals, collapse = " | ")
        return(out)
      } else {
        # list of objects: keep as compact JSON so nothing non-null is lost
        out[[prefix]] <- jsonlite::toJSON(x, auto_unbox = TRUE, null = "null")
        return(out)
      }
    }

    out
  }

  # --- fetch ---
  url <- paste0("https://clinicaltrials.gov/api/v2/studies/", nct_id)

  resp <- httr2::request(url) |>
    httr2::req_user_agent("tidyTrials (R)") |>
    httr2::req_error(is_error = function(r) httr2::resp_status(r) >= 400) |>
    httr2::req_perform()

  study <- jsonlite::fromJSON(httr2::resp_body_string(resp), simplifyVector = FALSE)

  # --- choose what to flatten ---
  if (is.null(module)) {
    target <- study
    prefix <- ""            # whole record
  } else {
    protocol <- study$protocolSection
    if (is.null(protocol) || !is.list(protocol)) {
      stop("No protocolSection found in the API response for this study.")
    }

    target <- protocol[[module]]
    if (is.null(target)) {
      available <- names(protocol)
      available <- available[!is.na(available) & nzchar(available)]
      stop(
        "Module '", module, "' not found in protocolSection.\n",
        "Available modules: ", paste(available, collapse = ", ")
      )
    }
    prefix <- module         # module-only record
  }

  # --- flatten ---
  flat <- flatten_any(target, prefix = prefix)

  # if nothing came through, return at least nct_id
  if (length(flat) == 0) {
    return(tibble::tibble(nct_id = nct_id))
  }

  # make column names readable + safe-ish
  nm <- names(flat)
  nm <- gsub("\\s+", "_", nm)
  nm <- gsub("[^A-Za-z0-9_.]+", "_", nm)
  names(flat) <- nm

  # keep the id column first
  flat <- c(list(nct_id = nct_id), flat)

  tibble::as_tibble(flat)
}




In [3]:
# all non-null fields from the whole study
ct_get_study_module_table("NCT04267848")

# only non-null fields inside one module
ct_get_study_module_table("NCT04267848", module = "statusModule")

nct_id,protocolSection.identificationModule.nctId,protocolSection.identificationModule.orgStudyIdInfo.id,protocolSection.identificationModule.secondaryIdInfos,protocolSection.identificationModule.organization.fullName,protocolSection.identificationModule.organization.class,protocolSection.identificationModule.briefTitle,protocolSection.identificationModule.officialTitle,protocolSection.statusModule.statusVerifiedDate,protocolSection.statusModule.overallStatus,⋯,protocolSection.referencesModule.references,protocolSection.ipdSharingStatementModule.ipdSharing,protocolSection.ipdSharingStatementModule.description,protocolSection.ipdSharingStatementModule.url,derivedSection.miscInfoModule.versionHolder,derivedSection.conditionBrowseModule.meshes,derivedSection.conditionBrowseModule.ancestors,derivedSection.interventionBrowseModule.meshes,derivedSection.interventionBrowseModule.ancestors,hasResults
<chr>,<chr>,<chr>,<json>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<json>,<chr>,<chr>,<chr>,<chr>,<json>,<json>,<json>,<json>,<chr>
NCT04267848,NCT04267848,NCI-2020-00751,"[{""id"":""NCI-2020-00751"",""type"":""REGISTRY"",""domain"":""CTRP (Clinical Trial Reporting Program)""},{""id"":""A081801"",""type"":""OTHER"",""domain"":""Alliance for Clinical Trials in Oncology""},{""id"":""A081801"",""type"":""OTHER"",""domain"":""CTEP""},{""id"":""U10CA180821"",""type"":""NIH"",""link"":""https://reporter.nih.gov/quickSearch/U10CA180821""}]",National Cancer Institute (NCI),NIH,"Testing the Addition of a Type of Drug Called Immunotherapy to the Usual Chemotherapy Treatment for Non-small Cell Lung Cancer, an ALCHEMIST Treatment Trial (Chemo-IO [ACCIO])",Integration of Immunotherapy Into Adjuvant Therapy for Resected NSCLC: ALCHEMIST Chemo-IO (ACCIO),2026-01,RECRUITING,⋯,"[{""pmid"":""33878954"",""type"":""DERIVED"",""citation"":""Sands JM, Mandrekar SJ, Kozono D, Oxnard GR, Hillman SL, Wigle DA, Govindan R, Carlisle J, Gray J, Salama JK, Raez L, Ganti A, Foster N, Malik S, Bradley J, Kelly K, Ramalingam SS, Stinchcombe TE. Integration of immunotherapy into adjuvant therapy for resected non-small-cell lung cancer: ALCHEMIST chemo-IO (ACCIO). Immunotherapy. 2021 Jun;13(9):727-734. doi: 10.2217/imt-2021-0019. Epub 2021 Apr 21.""}]",YES,"NCI is committed to sharing data in accordance with NIH policy. For more details on how clinical trial data is shared, access the link to the NIH data sharing policy page.",https://grants.nih.gov/policy/sharing.htm,2026-01-30,"[{""id"":""D002289"",""term"":""Carcinoma, Non-Small-Cell Lung""},{""id"":""D008175"",""term"":""Lung Neoplasms""}]","[{""id"":""D002283"",""term"":""Carcinoma, Bronchogenic""},{""id"":""D001984"",""term"":""Bronchial Neoplasms""},{""id"":""D012142"",""term"":""Respiratory Tract Neoplasms""},{""id"":""D013899"",""term"":""Thoracic Neoplasms""},{""id"":""D009371"",""term"":""Neoplasms by Site""},{""id"":""D009369"",""term"":""Neoplasms""},{""id"":""D008171"",""term"":""Lung Diseases""},{""id"":""D012140"",""term"":""Respiratory Tract Diseases""}]","[{""id"":""D013048"",""term"":""Specimen Handling""},{""id"":""D016190"",""term"":""Carboplatin""},{""id"":""D002945"",""term"":""Cisplatin""},{""id"":""C044245"",""term"":""1,2-diaminocyclohexaneplatinum II citrate""},{""id"":""D010984"",""term"":""Platinum""},{""id"":""D000093542"",""term"":""Gemcitabine""},{""id"":""D009682"",""term"":""Magnetic Resonance Spectroscopy""},{""id"":""D019370"",""term"":""Observation""},{""id"":""D017239"",""term"":""Paclitaxel""},{""id"":""D013660"",""term"":""Taxes""},{""id"":""C582435"",""term"":""pembrolizumab""},{""id"":""D000068437"",""term"":""Pemetrexed""}]","[{""id"":""D019411"",""term"":""Clinical Laboratory Techniques""},{""id"":""D019937"",""term"":""Diagnostic Techniques and Procedures""},{""id"":""D003933"",""term"":""Diagnosis""},{""id"":""D008919"",""term"":""Investigative Techniques""},{""id"":""D056831"",""term"":""Coordination Complexes""},{""id"":""D009930"",""term"":""Organic Chemicals""},{""id"":""D0176

nct_id,statusModule.statusVerifiedDate,statusModule.overallStatus,statusModule.expandedAccessInfo.hasExpandedAccess,statusModule.startDateStruct.date,statusModule.startDateStruct.type,statusModule.primaryCompletionDateStruct.date,statusModule.primaryCompletionDateStruct.type,statusModule.completionDateStruct.date,statusModule.completionDateStruct.type,statusModule.studyFirstSubmitDate,statusModule.studyFirstSubmitQcDate,statusModule.studyFirstPostDateStruct.date,statusModule.studyFirstPostDateStruct.type,statusModule.lastUpdateSubmitDate,statusModule.lastUpdatePostDateStruct.date,statusModule.lastUpdatePostDateStruct.type
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NCT04267848,2026-01,RECRUITING,FALSE,2020-06-16,ACTUAL,2027-01-31,ESTIMATED,2027-01-31,ESTIMATED,2020-02-12,2020-02-12,2020-02-13,ACTUAL,2026-01-29,2026-01-30,ACTUAL


## Jan 30

- created the API wrapper for the Metadata

In [ ]:
ct_fetch_study <- function(nct_id) {
  if (!requireNamespace("httr2", quietly = TRUE)) stop("Install httr2")
  if (!requireNamespace("jsonlite", quietly = TRUE)) stop("Install jsonlite")

  # clean the id
  nct_id <- toupper(trimws(nct_id))

  # basic format check: NCT + 8 digits is typical, but allow 6-12 digits to be safe
  if (!grepl("^NCT\\d{6,12}$", nct_id)) {
    stop("`nct_id` should look like 'NCT04267848'. You provided: ", shQuote(nct_id))
  }

  url <- paste0("https://clinicaltrials.gov/api/v2/studies/", nct_id)

  # perform request but handle 404 nicely
  req <- httr2::request(url) |>
    httr2::req_user_agent("tidyTrials (R)") |>
    httr2::req_perform()

  # If we got here, it's 2xx
  study <- jsonlite::fromJSON(httr2::resp_body_string(req), simplifyVector = FALSE)
  attr(study, "nct_id") <- nct_id
  attr(study, "url") <- url
  study
}

ct_metadata <- function(nct_id) {
  if (!requireNamespace("jsonlite", quietly = TRUE)) stop("Install jsonlite")
  if (!requireNamespace("tibble", quietly = TRUE)) stop("Install tibble")
  if (!requireNamespace("dplyr", quietly = TRUE)) stop("Install dplyr")
  if (!requireNamespace("httr2", quietly = TRUE)) stop("Install httr2")

  # ---- helpers ----
  is_empty <- function(x) {
    is.null(x) ||
      length(x) == 0 ||
      (is.character(x) && all(!nzchar(x))) ||
      (is.list(x) && length(x) == 0)
  }
  truncate_100 <- function(x) {
  x <- as.character(x)
  ifelse(nchar(x) > 100, paste0(substr(x, 1, 100), "..."), x)
  }


  leaf_type <- function(x) {
    if (is.null(x)) return("null")
    if (is.atomic(x) && !is.list(x)) {
      if (is.logical(x)) return("logical")
      if (is.integer(x)) return("integer")
      if (is.double(x))  return("double")
      if (is.character(x)) return("character")
      return(typeof(x))
    }
    if (is.list(x)) return("list")
    typeof(x)
  }

  flatten_to_rows <- function(x, prefix = "") {
  if (is_empty(x)) {
    return(dplyr::tibble(path = character(), value_type = character(), example_value = character()))
  }

  if (is.atomic(x) && !is.list(x)) {
    val <- x
    if (length(val) > 1) {
      val <- paste(as.character(val), collapse = " | ")
    } else {
      val <- as.character(val)[1]
    }

    if (is_empty(val)) {
      return(dplyr::tibble(path = character(), value_type = character(), example_value = character()))
    }

    return(
      dplyr::tibble(
        path = prefix,
        value_type = leaf_type(x),
        example_value = truncate_100(val)
      )
    )
  }

  if (is.list(x)) {
    if (!is.null(names(x)) && any(nzchar(names(x)))) {
      rows <- list()
      for (i in seq_along(x)) {
        nm <- names(x)[i]
        if (is.na(nm) || !nzchar(nm)) next
        key <- if (nzchar(prefix)) paste0(prefix, ".", nm) else nm
        rows[[length(rows) + 1]] <- flatten_to_rows(x[[i]], key)
      }
      return(dplyr::bind_rows(rows))
    }

    all_scalar <- all(vapply(x, function(z) is.atomic(z) && !is.list(z), logical(1)))
    if (all_scalar) {
      vals <- unlist(lapply(x, as.character), use.names = FALSE)
      vals <- vals[nzchar(vals)]
      if (length(vals) == 0) {
        return(dplyr::tibble(path = character(), value_type = character(), example_value = character()))
      }

      return(
        dplyr::tibble(
          path = prefix,
          value_type = "vector",
          example_value = truncate_100(paste(vals, collapse = " | "))
        )
      )
    } else {
      j <- jsonlite::toJSON(x, auto_unbox = TRUE, null = "null")
      return(
        dplyr::tibble(
          path = prefix,
          value_type = "list_of_objects",
          example_value = truncate_100(j)
        )
      )
    }
  }

  dplyr::tibble(path = character(), value_type = character(), example_value = character())
}



  module_from_path <- function(p) {
    parts <- strsplit(p, "\\.", fixed = FALSE)[[1]]
    if (length(parts) >= 2 && identical(parts[1], "protocolSection")) parts[2] else NA_character_
  }

  # ---- fetch with better 404 message ----
  nct_clean <- toupper(trimws(nct_id))
  url <- paste0("https://clinicaltrials.gov/api/v2/studies/", nct_clean)

  resp <- tryCatch(
    httr2::request(url) |>
      httr2::req_user_agent("tidyTrials (R)") |>
      httr2::req_perform(),
    error = function(e) {
      # surface status + URL
      msg <- conditionMessage(e)
      stop(
        "ClinicalTrials.gov returned an error for this study request.\n",
        "NCT ID: ", nct_clean, "\n",
        "URL: ", url, "\n",
        "Error: ", msg, "\n\n",
        "Common causes:\n",
        "- NCT ID is wrong (extra spaces, typo)\n",
        "- The study doesn't exist / is not available\n"
      )
    }
  )

  study <- jsonlite::fromJSON(httr2::resp_body_string(resp), simplifyVector = FALSE)

  protocol <- study$protocolSection
  if (is.null(protocol) || !is.list(protocol)) {
    stop("No protocolSection found in the API response for this study.")
  }

  modules <- names(protocol)
  modules <- modules[!is.na(modules) & nzchar(modules)]

  fields <- flatten_to_rows(protocol, prefix = "protocolSection") |>
    dplyr::mutate(module = vapply(path, module_from_path, character(1))) |>
    dplyr::filter(!is.na(module)) |>
    dplyr::arrange(module, path)

  list(
    nct_id = nct_clean,
    url = url,
    modules = modules,
    fields = tibble::as_tibble(fields)
  )
}


In [11]:
meta <- ct_metadata("NCT04267848")

meta$modules
meta$fields

[1] "identificationModule"       "statusModule"              
 [3] "sponsorCollaboratorsModule" "oversightModule"           
 [5] "descriptionModule"          "conditionsModule"          
 [7] "designModule"               "armsInterventionsModule"   
 [9] "outcomesModule"             "eligibilityModule"         
[11] "contactsLocationsModule"    "referencesModule"          
[13] "ipdSharingStatementModule"

path,value_type,example_value,module
<chr>,<chr>,<chr>,<chr>
protocolSection.armsInterventionsModule.armGroups,list_of_objects,"[{""label"":""Arm A (platinum doublet, observation)"",""type"":""ACTIVE_COMPARATOR"",""description"":""INITIAL ...",armsInterventionsModule
protocolSection.armsInterventionsModule.interventions,list_of_objects,"[{""type"":""PROCEDURE"",""name"":""Biospecimen Collection"",""description"":""Undergo blood sample collection""...",armsInterventionsModule
protocolSection.conditionsModule.conditions,vector,Lung Non-Small Cell Carcinoma | Lung Non-Small Cell Squamous Carcinoma | Lung Non-Squamous Non-Small...,conditionsModule
protocolSection.contactsLocationsModule.locations,list_of_objects,"[{""facility"":""Anchorage Associates in Radiation Medicine"",""status"":""SUSPENDED"",""city"":""Anchorage"",""s...",contactsLocationsModule
protocolSection.contactsLocationsModule.overallOfficials,list_of_objects,"[{""name"":""Jacob M Sands"",""affiliation"":""Alliance for Clinical Trials in Oncology"",""role"":""PRINCIPAL_...",contactsLocationsModule
protocolSection.descriptionModule.briefSummary,character,This phase III ALCHEMIST treatment trial tests the addition of pembrolizumab to usual chemotherapy f...,descriptionModule
protocolSection.descriptionModule.detailedDescription,character,PRIMARY OBJECTIVE: I. To compare the disease free survival (DFS) between Arm B versus (vs) Arm C in...,descriptionModule
protocolSection.designModule.designInfo.allocation,character,RANDOMIZED,designModule
protocolSection.designModule.designInfo.interventionModel,character,PARALLEL,designModule


In [17]:
suppressWarnings(devtools::load_all())
result <- get_study_module_table("NCT04267848")


ℹ Loading tidytrials


In [18]:
result

nct_id,protocolSection.identificationModule.nctId,protocolSection.identificationModule.orgStudyIdInfo.id,protocolSection.identificationModule.secondaryIdInfos,protocolSection.identificationModule.organization.fullName,protocolSection.identificationModule.organization.class,protocolSection.identificationModule.briefTitle,protocolSection.identificationModule.officialTitle,protocolSection.statusModule.statusVerifiedDate,protocolSection.statusModule.overallStatus,⋯,protocolSection.referencesModule.references,protocolSection.ipdSharingStatementModule.ipdSharing,protocolSection.ipdSharingStatementModule.description,protocolSection.ipdSharingStatementModule.url,derivedSection.miscInfoModule.versionHolder,derivedSection.conditionBrowseModule.meshes,derivedSection.conditionBrowseModule.ancestors,derivedSection.interventionBrowseModule.meshes,derivedSection.interventionBrowseModule.ancestors,hasResults
<chr>,<chr>,<chr>,<json>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<json>,<chr>,<chr>,<chr>,<chr>,<json>,<json>,<json>,<json>,<chr>
NCT04267848,NCT04267848,NCI-2020-00751,"[{""id"":""NCI-2020-00751"",""type"":""REGISTRY"",""domain"":""CTRP (Clinical Trial Reporting Program)""},{""id"":""A081801"",""type"":""OTHER"",""domain"":""Alliance for Clinical Trials in Oncology""},{""id"":""A081801"",""type"":""OTHER"",""domain"":""CTEP""},{""id"":""U10CA180821"",""type"":""NIH"",""link"":""https://reporter.nih.gov/quickSearch/U10CA180821""}]",National Cancer Institute (NCI),NIH,"Testing the Addition of a Type of Drug Called Immunotherapy to the Usual Chemotherapy Treatment for Non-small Cell Lung Cancer, an ALCHEMIST Treatment Trial (Chemo-IO [ACCIO])",Integration of Immunotherapy Into Adjuvant Therapy for Resected NSCLC: ALCHEMIST Chemo-IO (ACCIO),2026-01,RECRUITING,⋯,"[{""pmid"":""33878954"",""type"":""DERIVED"",""citation"":""Sands JM, Mandrekar SJ, Kozono D, Oxnard GR, Hillman SL, Wigle DA, Govindan R, Carlisle J, Gray J, Salama JK, Raez L, Ganti A, Foster N, Malik S, Bradley J, Kelly K, Ramalingam SS, Stinchcombe TE. Integration of immunotherapy into adjuvant therapy for resected non-small-cell lung cancer: ALCHEMIST chemo-IO (ACCIO). Immunotherapy. 2021 Jun;13(9):727-734. doi: 10.2217/imt-2021-0019. Epub 2021 Apr 21.""}]",YES,"NCI is committed to sharing data in accordance with NIH policy. For more details on how clinical trial data is shared, access the link to the NIH data sharing policy page.",https://grants.nih.gov/policy/sharing.htm,2026-01-30,"[{""id"":""D002289"",""term"":""Carcinoma, Non-Small-Cell Lung""},{""id"":""D008175"",""term"":""Lung Neoplasms""}]","[{""id"":""D002283"",""term"":""Carcinoma, Bronchogenic""},{""id"":""D001984"",""term"":""Bronchial Neoplasms""},{""id"":""D012142"",""term"":""Respiratory Tract Neoplasms""},{""id"":""D013899"",""term"":""Thoracic Neoplasms""},{""id"":""D009371"",""term"":""Neoplasms by Site""},{""id"":""D009369"",""term"":""Neoplasms""},{""id"":""D008171"",""term"":""Lung Diseases""},{""id"":""D012140"",""term"":""Respiratory Tract Diseases""}]","[{""id"":""D013048"",""term"":""Specimen Handling""},{""id"":""D016190"",""term"":""Carboplatin""},{""id"":""D002945"",""term"":""Cisplatin""},{""id"":""C044245"",""term"":""1,2-diaminocyclohexaneplatinum II citrate""},{""id"":""D010984"",""term"":""Platinum""},{""id"":""D000093542"",""term"":""Gemcitabine""},{""id"":""D009682"",""term"":""Magnetic Resonance Spectroscopy""},{""id"":""D019370"",""term"":""Observation""},{""id"":""D017239"",""term"":""Paclitaxel""},{""id"":""D013660"",""term"":""Taxes""},{""id"":""C582435"",""term"":""pembrolizumab""},{""id"":""D000068437"",""term"":""Pemetrexed""}]","[{""id"":""D019411"",""term"":""Clinical Laboratory Techniques""},{""id"":""D019937"",""term"":""Diagnostic Techniques and Procedures""},{""id"":""D003933"",""term"":""Diagnosis""},{""id"":""D008919"",""term"":""Investigative Techniques""},{""id"":""D056831"",""term"":""Coordination Complexes""},{""id"":""D009930"",""term"":""Organic Chemicals""},{""id"":""D0176

In [19]:
suppressWarnings(devtools::load_all())
meta <- get_metadata("NCT04267848")

ℹ Loading tidytrials


In [24]:
meta$modules


[1] "identificationModule"       "statusModule"              
 [3] "sponsorCollaboratorsModule" "oversightModule"           
 [5] "descriptionModule"          "conditionsModule"          
 [7] "designModule"               "armsInterventionsModule"   
 [9] "outcomesModule"             "eligibilityModule"         
[11] "contactsLocationsModule"    "referencesModule"          
[13] "ipdSharingStatementModule"